<h1>template notebook</h1>

# import libraries

In [13]:
#defaul libraries
#https://docs.python.org/ja/
import os
import sys
import io

import math
import random
import pprint
import time
import datetime
import typing
import json
import glob
import requests
import warnings
import gc
from pprint import pprint
import re

import numpy as np #https://numpy.org/
import pandas as pd #https://pandas.pydata.org/
import sklearn #https://scikit-learn.org/stable/

import matplotlib.pyplot as plt #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

import seaborn as sns
sns.set()

from tqdm import tqdm #https://tqdm.github.io/

import torch #https://pytorch.org/
import transformers #https://huggingface.co/transformers/

import torchvision
import torchtext

In [14]:
print(os.curdir)
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.
./__notebook_source__.ipynb


# Configuration

In [15]:
class CFG():
    
    data_path="../input/feedback-prize-2021/"
    debug=True
    seed=0

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    batch_size=1#32
    epochs=30
    learning_rate=0.1
    kFold=5

    amp=False


    #高速化関連
    #https://qiita.com/sugulu_Ogawa_ISID/items/62f5f7adee083d96a587

    #GPU 遅くなるらしい↓
    torch.backends.cudnn.deterministic = True

    #イテレーションごとのnnの順伝搬および誤差関数の 計算手法がある程度一定であれば、torch.backends.cudnn.benchmark = Trueで GPU での計算が高速化
    torch.backends.cudnn.benchmark = False


def set_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(CFG.seed)

print(CFG.device)
if torch.cuda.is_available():
    torch.cuda.set_device(CFG.device)
    print(torch.cuda.current_device())
    print(torch.cuda.get_device_name())

if CFG.debug:
    CFG.epochs=5

cpu


# Color

In [16]:
def color(string,fg='DEFAULT',bg='DEFAULT',fg_rgb=None,bg_rgb=None,style='END'):
    colors=['BLACK','RED','GREEN','YELLOW','BLUE','PURPLE','CYAN','WHITE','8','DEFAULT']
    styles=['END','BOLD','2','3','UNDERLINE','5','6','REVERSE','INVISIBLE','9']

    fg=f'\033[3{colors.index(fg)}m'
    bg=f'\033[4{colors.index(bg)}m'
    style=f'\033[0{styles.index(style)}m'

    if fg_rgb:fg=f"\033[38;2;{fg_rgb[0]};{fg_rgb[1]};{fg_rgb[2]}m"
    if bg_rgb:bg=f"\033[48;2;{bg_rgb[0]};{bg_rgb[1]};{bg_rgb[2]}m"

    return style+fg+bg+str(string)+'\033[0m'

In [17]:
print(color("color test","GREEN",style='BOLD'))
print(color("color test","RED","BLUE",style='UNDERLINE'))
print(color("color test","CYAN","WHITE",style='INVISIBLE'))
print(color("color test","CYAN",style='REVERSE'))
print(color("aaa",fg_rgb=(150,150,255)))
# print(color("bbb",fg_rgb=(0,250,50),bg_rgb=(0,0,255)))
# print(color("ccc"))
# print(color('style',bg="RED",fg='BLUE'))
# print('\033[07m'+"Aa")


color test
color test
color test
color test
aaa


In [18]:
def what_is(obj):
    print("type:",type(obj))
    print("-"*40+"help"+"-"*40)
    print(help(obj))
    print("-"*40+"dir"+"-"*40)
    print(dir(obj))

In [19]:
target_columns=['target']
def feature_columns(df):
    for col in df.columns:
        if col in target_columns:df=df.drop(columns=col)
    return  df.columns
def numerical_columns(df):
    return df[feature_columns(df)].select_dtypes(include=['int64','float64']).columns
def categorical_columns(df):
    return df[feature_columns(df)].select_dtypes(exclude=['int64','float64']).columns

read data

In [20]:

train=pd.read_csv(os.path.join(CFG.data_path,"train.csv"),index_col='id')
# test=pd.read_csv(os.path.join(CFG.data_path,"test.csv"),index_col='id')
sample_submission=pd.read_csv(os.path.join(CFG.data_path,"sample_submission.csv"),index_col='id')

if CFG.debug:
    #train=train[:len(train)//50]
    train=train.sample(frac=0.1,random_state=CFG.seed)

# df=pd.concat([train,test])
# train=df[:len(train)].copy()
# test=df[len(train):].copy()

# data={
#     'train':pd.DataFrame(columns=['feature','target']),
#     'val':pd.DataFrame({'feature':val.feature,'target':val['target']}),
#     'test':pd.DataFrame(test),
# }



In [21]:
train

,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
id,,,,,,,
2C27E003116B,1.622648e+12,408.0,529.0,Think of all the money people would save becau...,Claim,Claim 1,71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 8...
7BEB99575D62,1.615844e+12,135.0,198.0,Popular vote is a more efficent way to vote fo...,Claim,Claim 1,21 22 23 24 25 26 27 28 29 30 31 32
CE4EFA6441C4,1.614611e+12,935.0,1460.0,"or most problems, the town pays people or a bu...",Evidence,Evidence 2,168 169 170 171 172 173 174 175 176 177 178 17...
0EE6F614F2F0,1.618171e+12,110.0,180.0,Seeking more than one opinion can provide som...,Position,Position 1,20 21 22 23 24 25 26 27 28 29 30
6E72E9EADF75,1.614702e+12,0.0,326.0,""" What's wrong with the Electoral College?"" To...",Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
...,...,...,...,...,...,...,...
DA8115F12B50,1.622570e+12,553.0,615.0,reduced the amount of greenhouse gasses in Ea...,Claim,Claim 1,91 92 93 94 95 96 97 98 99
C22959B3262C,1.618456e+12,1124.0,1745.0,Why would some of the students choose student-...,Rebuttal,Rebuttal 1,194 195 196 197 198 199 200 201 202 203 204 20...
937CDD403137,1.620761e+12,0.0,37.0,Driveless cars can be very dangerous.,Position,Position 1,0 1 2 3 4 5


In [22]:
class datapath:
    def __init__(self,path):
        self.data_path=path
    def __getitem__(self,idx):
        return self.data_path+str(idx)+'.png'
    
data={
    'train':datapath('/kaggle/working/train/'),
    'test':datapath('/kaggle/working/test/'),
}

submission

In [23]:
import pandas as pd

In [24]:
submission=pd.DataFrame(index=pd.Series(name='row_id',dtype='int'),columns=['idx','target'])


submission.idx=range(5)
submission.target=[1,0,0,1,1]

#submission.sort_index(inplace=True)

#submission.to_csv("submission.csv",index=idx)

submission.head()

,idx,target
row_id,,
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
